# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('../dataset/cifar/train/*.png')[:100]
test_paths = glob('../dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Training

http://localhost:6006

In [15]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

Epoch 1/5
3/3 [==============================] - 1s 113ms/step - loss: 2.2888 - accuracy: 0.1324 - val_loss: 2.3394 - val_accuracy: 0.0729
Epoch 2/5
3/3 [==============================] - 0s 126ms/step - loss: 2.2975 - accuracy: 0.0417 - val_loss: 2.3245 - val_accuracy: 0.0729
Epoch 3/5
3/3 [==============================] - 0s 77ms/step - loss: 2.2689 - accuracy: 0.1500 - val_loss: 2.3330 - val_accuracy: 0.0729
Epoch 4/5
3/3 [==============================] - 0s 118ms/step - loss: 2.3334 - accuracy: 0.0625 - val_loss: 2.3498 - val_accuracy: 0.0729
Epoch 5/5
3/3 [==============================] - 0s 89ms/step - loss: 2.2877 - accuracy: 0.1324 - val_loss: 2.3445 - val_accuracy: 0.0729


# Load Checkpoint

In [ ]:
model.load_weights('checkpoints') # checkpoint가 담긴 폴더를 지정해주면 checkpoint도 불러올 수 있음

# Saving Model

In [16]:
save_path = 'my_model.h5'

In [17]:
model.save(save_path, include_optimizer=True) # optimizer도 저장해서 compile 다시 안불러와도 됨

In [18]:
model = tf.keras.models.load_model('my_model.h5')

# Saving Model - 2

In [19]:
model.save_weights('model_weights.h5') # weights만 저장

In [20]:
with open('model_architecture.json', 'w') as f: # json으로 저장
    f.write(model.to_json())

In [22]:
from tensorflow.keras.models import model_from_json

In [23]:
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

In [24]:
model.load_weights('model_weights.h5')

# model.h5 들여다보기

In [25]:
import h5py

In [26]:
model_file = h5py.File('my_model.h5', 'r+')

In [27]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [28]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [36]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [33]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [34]:
weight = np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

In [35]:
weight

array([[[[-8.34956244e-02,  5.92730567e-02, -3.81341614e-02,
          -6.90699741e-02,  9.37411934e-02, -7.37203285e-02,
          -1.19819529e-02, -7.91768730e-03,  7.06749111e-02,
           1.13666885e-01, -8.64268914e-02,  1.75881349e-02,
           3.26321274e-02, -1.04433395e-01,  2.24545458e-03,
          -8.00712630e-02,  6.14075698e-02,  8.95054415e-02,
           3.33223343e-02, -2.39734780e-02,  3.61158163e-06,
          -1.00624733e-01, -1.65185947e-02, -3.46128270e-02,
           2.27606762e-02,  5.14242239e-02, -1.36757284e-01,
          -2.48077065e-02, -9.66233164e-02,  1.19511217e-01,
          -4.60521411e-03, -3.58056389e-02],
         [ 7.89053515e-02, -9.01929848e-03, -1.03441440e-02,
          -3.56193595e-02, -2.06960533e-02, -5.72380275e-02,
          -3.07180788e-02,  9.54111367e-02, -7.29717091e-02,
           7.91502818e-02, -5.21331690e-02, -2.63240729e-02,
          -1.40720665e-01,  6.85756952e-02, -1.19495578e-02,
          -1.16902053e-01,  6.97265118e-